In [1]:
# Dependencies and Setups
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import gkey
gmaps.configure(api_key=gkey)


In [2]:
#Create DataFrame from WeatherPy Output File (CSV)

cities_df = pd.read_csv("../WeatherPy/File Output/cities.csv")
cities_df.head()

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,cidreira,-30.18,-50.21,62.636,83.0,10.0,7.58,BR,08-26-2020
1,gedo,9.02,37.45,50.198,100.0,11.0,0.55,ET,08-26-2020
2,esperance,-33.87,121.90,46.400,100.0,0.0,4.60,AU,08-26-2020
3,bredasdorp,-34.53,20.04,57.200,71.0,14.0,4.10,ZA,08-26-2020
4,bahia blanca,-38.72,-62.27,52.520,86.0,100.0,4.08,AR,08-26-2020


In [3]:
#Create gmaps figure

fig = gmaps.figure()

In [4]:
#Create heatmap layer

locations = cities_df[["Lat","Lon"]]
humidity = cities_df["Humidity"]

heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False,
                             max_intensity = 100, point_radius=5)

In [5]:
#Add heatmap layer

fig.add_layer(heat_layer)

In [6]:
#Display gmaps figure

fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Filter cities with ideal temp, wind, and cloud conditions & create new DataFrame

ideal_cities = cities_df.loc[(cities_df["Max Temp"]<80) & (cities_df["Max Temp"]>70) & (cities_df["Wind Speed"]<10)
                                & (cities_df["Humidity"]<40) & (cities_df["Cloudiness"]<70), :]
ideal_cities_df = pd.DataFrame(ideal_cities)
ideal_cities_df

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
31,bethel,41.37,-73.41,75.992,31.0,1.0,4.60,US,08-26-2020
169,yinchuan,38.47,106.27,72.536,39.0,0.0,2.20,CN,08-26-2020
243,brewster,41.40,-73.62,77.000,31.0,1.0,4.60,US,08-26-2020
251,palmer,42.16,-72.33,73.400,33.0,40.0,7.20,US,08-26-2020
518,tete,-16.16,33.59,79.394,36.0,0.0,1.95,MZ,08-26-2020


In [8]:
#Add "Hotel Name" column

ideal_cities_df["Hotel Name"] = ""
ideal_cities_df

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
31,bethel,41.37,-73.41,75.992,31.0,1.0,4.60,US,08-26-2020,
169,yinchuan,38.47,106.27,72.536,39.0,0.0,2.20,CN,08-26-2020,
243,brewster,41.40,-73.62,77.000,31.0,1.0,4.60,US,08-26-2020,
251,palmer,42.16,-72.33,73.400,33.0,40.0,7.20,US,08-26-2020,
518,tete,-16.16,33.59,79.394,36.0,0.0,1.95,MZ,08-26-2020,


In [9]:
#Question: Ask about how iterrows works

for index, row in ideal_cities_df.iterrows(): 
    print(row["City"])

bethel
yinchuan
brewster
palmer
tete


In [10]:
#Use Google Places API to find hotels near all ideal vacation spots

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
hotel_list = []
hotel_dict = {}

for index,row in ideal_cities_df.iterrows(): 
    try:
        lat = row["Lat"]
        long = row["Lon"]
        location = f"{lat},{long}"

        params = {
            "location": location, 
            "radius": 5000,
            "type": "hotel",
            "key": gkey}

        response = requests.get(base_url, params=params).json()

        name = response["results"][0]["name"]
   
    except:
        print("No hotel information available for this city")
        name = np.NaN
    
    ideal_cities_df.loc[index,"Hotel Name"] = name

No hotel information available for this city


In [11]:
# Display ideal cities DataFrame with Hotel Names added

ideal_cities_df = ideal_cities_df.dropna(how="any")
ideal_cities_df

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
31,bethel,41.37,-73.41,75.992,31.0,1.0,4.60,US,08-26-2020,Danbury
169,yinchuan,38.47,106.27,72.536,39.0,0.0,2.20,CN,08-26-2020,Yinchuan
243,brewster,41.40,-73.62,77.000,31.0,1.0,4.60,US,08-26-2020,Carmel Hamlet
518,tete,-16.16,33.59,79.394,36.0,0.0,1.95,MZ,08-26-2020,Tete


In [12]:
#Create marker layer with info box

locations = ideal_cities_df[["Lat","Lon"]]

info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in ideal_cities_df.iterrows()]

marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)

In [13]:
#Add markery layer to gmaps figure

fig.add_layer(marker_layer)

In [14]:
#Question: Why do multiple maps show on same window?
#Display gmaps figure 

fig

Figure(layout=FigureLayout(height='420px'))